In [9]:
from pathlib import Path
from itertools import chain
from pandas import DataFrame, Series
from rolling_pin.conform_etl import ConformETL

In [10]:
exclude_re = '.(git|mypy|pytest)|__(pycache|mypy)__'
source_dir = '/home/ubuntu/rolling-pin/resources/conform_repo'
target_dir = '/tmp/repo'
config =dict(
    source_rules=[
        dict(path=source_dir, include='README|LICENSE'),
        dict(path=source_dir, include='pdm|pyproject'),
        dict(path=source_dir + '/python', include=r'\.py$', exclude=r'_test'),
    ],
    rename_rules=[
        dict(regex='/source', replace='/target'),
        dict(regex='FAKE-|fake-', replace=''),
        dict(regex='/docker', replace=''),
        dict(regex='/python', replace=''),
        dict(regex='/pdm.lock', replace='/.pdm.lock'),
    ],
    group_rules=[
        dict(name='init', regex="__init__.py$"),
        dict(name='test', regex="_test"),
        dict(name='resource', regex="/resources"),
    ],
    line_rules=[
        dict(group='init', include='baz', exclude='test|ignore'),
    ],
)
    
d = ConformETL(**config)
d.to_html()

In [11]:
def buffer_filepath(data):
    data = data.copy()
    output = data \
        .apply(lambda x: Path(x).parts[1:-1]) \
        .apply(lambda x: list(chain(*zip(['/'] * len(x), x)))) \
        .tolist()
    output = DataFrame(output)
    output['sep'] = '/'
    output['filename'] = data.apply(lambda x: Path(x).name)
    output = output.fillna('')
    output.columns = list(range(output.shape[1]))
    output = output.to_string(index=False, header=False).split('\n')
    output = Series(output)
    return output

data = d._data.copy()
data.source = buffer_filepath(data.source)
data.target = buffer_filepath(data.target)
data['==>'] = '==>'
data = data[['source', '==>', 'target', 'groups']]
data.rename(lambda x: x.upper(), axis=1, inplace=True)
x = data.to_string(index=False, max_colwidth=150, col_space=[80, 10, 80, 20], justify='left')
print(x)

SOURCE                                                                                        ==>        TARGET                                                                           GROUPS              
/ home / ubuntu / rolling-pin / resources / conform_repo                /        FAKE-LICENSE ==>        / home / ubuntu / rolling-pin / resources / conform_repo       /        LICENSE        [resource]    
/ home / ubuntu / rolling-pin / resources / conform_repo                /         FAKE-README ==>        / home / ubuntu / rolling-pin / resources / conform_repo       /         README        [resource]    
/ home / ubuntu / rolling-pin / resources / conform_repo / docker       /       fake-pdm.lock ==>        / home / ubuntu / rolling-pin / resources / conform_repo       /      .pdm.lock        [resource]    
/ home / ubuntu / rolling-pin / resources / conform_repo / docker       /       fake-pdm.toml ==>        / home / ubuntu / rolling-pin / resources / conform_repo       /   